In [1]:
dataset_id = "bollywood_modeled"
!bq --location=US mk --dataset {dataset_id} 

Dataset 'starry-center-266501:bollywood_modeled' successfully created.


In [49]:
%%bigquery
CREATE TABLE bollywood_modeled.bollywoodTitles 
AS
SELECT DISTINCT Title title, Release_Month MMM, CAST(Release_Date AS INT64) AS DD, Year YYYY, CAST(Highest_Grosser_By_Year_in_crores_ AS NUMERIC) croresGrossed
FROM bollywood_staging.bollywood

""


## Sample bollywoodTitles

In [51]:
%%bigquery
SELECT * 
FROM bollywood_modeled.bollywoodTitles
WHERE MMM != 'None'
AND croresGrossed IS NOT NULL

,title,MMM,DD,YYYY,croresGrossed
0,Dabangg,SEP,10,2010,250.000000000
1,Ek Tha Tiger,AUG,15,2012,334.390000000
2,PK,DEC,19,2014,832.000000000
3,Dhoom 3,DEC,20,2013,589.200000000
4,Tiger Zinda Hai,DEC,22,2017,565.100000000
5,Dangal,DEC,23,2016,2024.000000000
6,Bajrangi Bhaijaan,JUL,24,2015,969.060000000
7,3 Idiots,DEC,25,2009,459.960000000
8,Ghajini,DEC,26,2008,232.000000000
9,Sanju,JUN,29,2018,586.860000000


In [52]:
%%bigquery
SELECT COUNT(*)
FROM bollywood_modeled.bollywoodTitles

,f0_
0,9660


In [53]:
%%bigquery
SELECT DISTINCT COUNT(title)
FROM bollywood_modeled.bollywoodTitles

,f0_
0,9660


## Title is the PK for this table

In [44]:
%%bigquery
CREATE TABLE bollywood_modeled.bollywoodDirects
AS
SELECT DISTINCT Director director, Title title
FROM bollywood_staging.bollywood

""


In [46]:
%%bigquery
SELECT *
FROM bollywood_modeled.bollywoodDirects
LIMIT 5

,director,title
0,Jal,Dr. Z
1,Anay,Happy Husbands
2,Baby,Bhayaanak Mahal
3,Baby,House No. 13
4,Bapu,Paramaatma


In [48]:
%%bigquery
SELECT COUNT(*)
FROM bollywood_modeled.bollywoodDirects

,f0_
0,9613


In [54]:
%%bigquery
SELECT COUNT (*)
FROM
  (SELECT DISTINCT director, title
  FROM bollywood_modeled.bollywoodDirects)

,f0_
0,9613


## Director and title make up a composite primary key

In [56]:
%%bigquery
SELECT d.title 
FROM bollywood_modeled.bollywoodDirects d
WHERE d.title NOT IN
  (SELECT DISTINCT t.title 
   FROM bollywood_modeled.bollywoodTitles t)

,title


## Referential integrity is maintained between parent table bollywoodTitles and child table bollywoodDirects on the title field

In [62]:
%%bigquery
CREATE TABLE bollywood_modeled.bollywoodGenres
AS
SELECT DISTINCT Title title, Genre genre
FROM bollywood_staging.bollywood
WHERE genre != 'None'

""


In [63]:
%%bigquery
SELECT * 
FROM bollywood_modeled.bollywoodGenres
LIMIT 5

,title,genre
0,Haqeeqat,war
1,Border,war
2,1971,war
3,Param Vir Chakra,war
4,Yeh Saali Zindagi,cult


In [64]:
%%bigquery
SELECT COUNT (*)
FROM bollywood_modeled.bollywoodGenres

,f0_
0,9449


In [16]:
%%bigquery
SELECT DISTINCT COUNT(*)
FROM
  (SELECT DISTINCT title, genre
  FROM bollywood_modeled.bollywoodGenres)

,f0_
0,9449


## title and genre make up a composite primary key

In [69]:
%%bigquery
SELECT g.title
FROM bollywood_modeled.bollywoodGenres g
WHERE g.title NOT IN
  (SELECT t.title
   FROM bollywood_modeled.bollywoodTitles t)

,title


## The referential integrity between parent table bollywoodTitles and child table bollywoodGenres is maintained on field title

In [22]:
%%bigquery
CREATE TABLE bollywood_modeled.bollywoodCast 
AS
SELECT Title title, b.Cast names
FROM bollywood_staging.bollywood b
WHERE b.Cast != "unknown"

""


In [23]:
%%bigquery
SELECT * 
FROM bollywood_modeled.bollywoodCast
LIMIT 5

,title,names
0,D,"randeep,chunky,koppikar,isha,pandey,hooda"
1,W,"sonal,danish,giani,pandor,leslie,meer,abhey,al..."
2,X,"kapoor,qureshi,huma,rajat"
3,3G,"sonal,neil,chauhan,nitin,mukesh"
4,99,"vinod,khan,soha,khemu,ali,boman,simone,khanna,..."


In [24]:
%%bigquery
SELECT COUNT (*)
FROM bollywood_modeled.bollywoodCast

,f0_
0,8989


In [27]:
%%bigquery
SELECT DISTINCT COUNT(*)
FROM
  (SELECT DISTINCT title, names
  FROM bollywood_modeled.bollywoodCast)

,f0_
0,8987


## title and names should be a composite primary key but a transformation needs to be done

In [30]:
%%bigquery
SELECT title 
FROM bollywood_modeled.bollywoodCast c
WHERE c.title NOT IN
  (SELECT title 
   FROM bollywood_modeled.bollywoodTitles)

,title


## Referential integrity is maintained between parent table "bollywoodTitles" and child table "bollywoodCast" on field "title"

In [65]:
%%bigquery
SELECT primaryTitle, startYear
FROM imdb_modeled.title_Beam_DF 
JOIN bollywood_modeled.bollywoodTitles 
ON title = primaryTitle 
AND startYear = YYYY

,primaryTitle,startYear
0,Jurm,2005
1,Bhule Bhatke,1952
2,Dayavan,1988
3,Don Muthu Swami,2008
4,Phir Bhi Apna Hai,1946
...,...,...
6177,Meena Bazaar,1950
6178,Hamari Beti,1950
6179,Rock On!!,2008
6180,Bhakta Prahlad,1932


In [74]:
%%bigquery
CREATE TABLE bollywood_modeled.bollywoodActs
AS
SELECT Name actor, Height_in_cm_ height_cm, "actress" AS job 
FROM bollywood_staging.bollywoodActress
UNION ALL
SELECT Name actor, Height_in_cm_ height_cm, "actor" AS job 
FROM bollywood_staging.bollywoodActors

""


In [75]:
%%bigquery
SELECT *
FROM bollywood_modeled.bollywoodActs
LIMIT 5

,actor,height_cm,job
0,Aamir Khan,163,actor
1,Himesh Reshamiya,163,actor
2,Kamal Haasan,165,actor
3,Kunal Khemu,166,actor
4,Shahid Kapoor,167,actor


In [76]:
%%bigquery
SELECT COUNT(*)
FROM bollywood_modeled.bollywoodActs

,f0_
0,273


In [77]:
%%bigquery
SELECT DISTINCT COUNT(actor)
FROM bollywood_modeled.bollywoodActs

,f0_
0,273


## Actor is a primary key

## Apache Beam Transforms

In [1]:
%run bollywoodCast_beam.py

/home/jupyter/venv/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:1466: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []
INFO:apache_beam.runners.direct.direct_runner:Running pipeline with DirectRunner.
INFO:apache_beam.internal.gcp.auth:Setting socket default timeout to 60 seconds.
INFO:apache_beam.internal.gcp.auth:socket default timeout is 60.0 seconds.
INFO:oauth2client.transport:Attempting refresh to obtain initial access_token
INFO:apache_beam.io.gcp.bigquery_tools:Using location 'US' from table <TableReference
 datasetId: 'bollywood_modeled'
 projectId: 'starry-center-266501'
 tableId: 'bollywoodCast'> referenced by query SELECT * FROM bollywood_modeled.bollywoodCast limit 100
INFO:apache_beam.io.gcp.bigquery_tools:Created table starry-center-266501.bollywood_modeled.bollywoodCast_Beam with schema <TableSchema
 fields: [<Ta

In [ ]:
%run bollywoodCast_beam_dataflow

/home/jupyter/venv/lib/python3.7/site-packages/apache_beam/runners/dataflow/dataflow_runner.py:788: BeamDeprecationWarning: BigQuerySink is deprecated since 2.11.0. Use WriteToBigQuery instead.
  kms_key=transform.kms_key))


## Format the dates in the bollywoodTitles table

In [14]:
%run bollywoodTitles_beam.py

/home/jupyter/venv/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:1466: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []
INFO:apache_beam.runners.direct.direct_runner:Running pipeline with DirectRunner.
INFO:apache_beam.internal.gcp.auth:Setting socket default timeout to 60 seconds.
INFO:apache_beam.internal.gcp.auth:socket default timeout is 60.0 seconds.
INFO:oauth2client.transport:Attempting refresh to obtain initial access_token
INFO:apache_beam.io.gcp.bigquery_tools:Using location 'US' from table <TableReference
 datasetId: 'bollywood_modeled'
 projectId: 'starry-center-266501'
 tableId: 'bollywoodTitles'> referenced by query SELECT * FROM bollywood_modeled.bollywoodTitles limit 100
INFO:apache_beam.io.gcp.bigquery_tools:Created table starry-center-266501.bollywood_modeled.bollywoodTitles_Beam with schema <TableSchema
 fields

In [1]:
%run bollywoodTitles_beam_dataflow.py

/home/jupyter/venv/lib/python3.7/site-packages/apache_beam/runners/dataflow/dataflow_runner.py:788: BeamDeprecationWarning: BigQuerySink is deprecated since 2.11.0. Use WriteToBigQuery instead.
  kms_key=transform.kms_key))


In [12]:
%run bollywoodDirects_beam.py

/home/jupyter/venv/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:1466: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []
INFO:apache_beam.runners.direct.direct_runner:Running pipeline with DirectRunner.
INFO:apache_beam.io.gcp.bigquery_tools:Using location 'US' from table <TableReference
 datasetId: 'bollywood_modeled'
 projectId: 'starry-center-266501'
 tableId: 'bollywoodDirects'> referenced by query SELECT * FROM bollywood_modeled.bollywoodDirects limit 100
INFO:apache_beam.io.gcp.bigquery_tools:Created table starry-center-266501.bollywood_modeled.bollywoodDirects_Beam with schema <TableSchema
 fields: [<TableFieldSchema
 fields: []
 mode: 'NULLABLE'
 name: 'title'
 type: 'STRING'>, <TableFieldSchema
 fields: []
 mode: 'NULLABLE'
 name: 'director'
 type: 'STRING'>]>. Result: <Table
 creationTime: 1587881380008
 etag: 'ZxNzohmu1

In [ ]:
%run bollywoodDirects_beam_dataflow.py

/home/jupyter/venv/lib/python3.7/site-packages/apache_beam/runners/dataflow/dataflow_runner.py:788: BeamDeprecationWarning: BigQuerySink is deprecated since 2.11.0. Use WriteToBigQuery instead.
  kms_key=transform.kms_key))


In [1]:
%run bollywoodGenres_beam.py

/home/jupyter/venv/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:1466: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []
INFO:apache_beam.runners.direct.direct_runner:Running pipeline with DirectRunner.
INFO:apache_beam.internal.gcp.auth:Setting socket default timeout to 60 seconds.
INFO:apache_beam.internal.gcp.auth:socket default timeout is 60.0 seconds.
INFO:oauth2client.transport:Attempting refresh to obtain initial access_token
INFO:apache_beam.io.gcp.bigquery_tools:Using location 'US' from table <TableReference
 datasetId: 'bollywood_modeled'
 projectId: 'starry-center-266501'
 tableId: 'bollywoodGenres'> referenced by query SELECT * FROM bollywood_modeled.bollywoodGenres limit 100
INFO:apache_beam.io.gcp.bigquery_tools:Created table starry-center-266501.bollywood_modeled.bollywoodGenres_Beam with schema <TableSchema
 fields

In [ ]:
%run bollywoodGenres_beam_dataflow.py

/home/jupyter/venv/lib/python3.7/site-packages/apache_beam/runners/dataflow/dataflow_runner.py:788: BeamDeprecationWarning: BigQuerySink is deprecated since 2.11.0. Use WriteToBigQuery instead.
  kms_key=transform.kms_key))


## Verify Beam result tables have primary keys

In [1]:
%%bigquery
SELECT COUNT (*)
FROM bollywood_modeled.bollywoodCast_Beam_DF

,f0_
0,71141


In [12]:
%%bigquery
SELECT DISTINCT COUNT (*)
FROM
  (SELECT (title, name)
  FROM bollywood_modeled.bollywoodCast_Beam_DF)

,f0_
0,71141


### Title and name form a composite primary key for bollywoodCast_Beam_DF

In [1]:
%%bigquery
SELECT COUNT (*)
FROM bollywood_modeled.bollywoodTitles_Beam_DF

,f0_
0,9660


In [3]:
%%bigquery
SELECT DISTINCT COUNT(title)
FROM bollywood_modeled.bollywoodTitles_Beam_DF

,f0_
0,9660


### Title is a primary key for bollywoodTitles_Beam_DF

In [4]:
%%bigquery
SELECT COUNT (*)
FROM bollywood_modeled.bollywoodDirects_Beam_DF

,f0_
0,9840


In [1]:
%%bigquery
SELECT DISTINCT COUNT (*)
FROM
  (SELECT director, title
  FROM bollywood_modeled.bollywoodDirects_Beam_DF)

,f0_
0,9840


### Director and title form a composite primary key for bollywoodDirects_Beam_DF

In [6]:
%%bigquery
SELECT COUNT (*)
FROM bollywood_modeled.bollywoodGenres_Beam_DF

,f0_
0,7574


In [7]:
%%bigquery
SELECT DISTINCT COUNT(title)
FROM bollywood_modeled.bollywoodGenres_Beam_DF

,f0_
0,7574


### Title is a primary key for bollywoodGenres_Beam_DF

### Verify referential integrity between bollywoodTitles_Beam_DF and child tables

In [21]:
%%bigquery
SELECT g.title 
FROM bollywood_modeled.bollywoodGenres_Beam_DF g
WHERE g.title NOT IN
  (SELECT title FROM bollywood_modeled.bollywoodTitles_Beam_DF)

,title


bollywoodTitles_Beam_DF is the parent table to bollywoodGenres_Beam_DF

In [23]:
%%bigquery
SELECT d.title
FROM bollywood_modeled.bollywoodDirects_Beam_DF d
WHERE d.title NOT IN
  (SELECT title FROM bollywood_modeled.bollywoodTitles_Beam_DF)
    

,title


bollywoodTitles_Beam_DF is the parent table to bollywoodDirects_Beam_DF

In [24]:
%%bigquery
SELECT title 
FROM bollywood_modeled.bollywoodCast_Beam_DF
WHERE title NOT IN
  (SELECT title FROM bollywood_modeled.bollywoodTitles_Beam_DF)

,title


bollywoodTitles_Beam_DF is the parent table to bollywoodCast_Beam_DF